# Exploration

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

raw_data = pd.read_csv('raw_merged.csv')


/var/folders/2k/zr998sp53tl9pl2r7nkqy47h0000gn/T/ipykernel_72680/2575136022.py:5: DtypeWarning: Columns (1,7,21,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('raw_merged.csv')


In [16]:
#split data insto 70 train, 15 val, 15 test
train_data, temp_data = train_test_split(raw_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (2164743, 31)
Validation data shape: (463874, 31)
Test data shape: (463874, 31)


In [20]:

print(train_data['overall_cond'].value_counts())
print('NA COUNTS: ', train_data.isna().sum())
print('num sam ids: ', train_data['sam_id'].nunique())


overall_cond
A - Average       1798955
G - Good           111181
EX - Excellent      16377
E - Excellent       14467
VG - Very Good       7101
F - Fair              689
P - Poor               12
Name: count, dtype: int64
NA COUNTS:  Unnamed: 0                 0
over_5               1976368
year                       0
full_address               0
sam_id                     0
building_id                0
ward_id                    0
parcel_num                 0
case_created_date          5
last_case_update           0
targeted_deadline     303630
close_date            152674
case_met_deadline      11900
case_status            11893
closure_reason         11893
case_title             11893
case_subject           11893
case_reason            11893
case_type              11893
case_department        11893
p_id                  191098
cm_id                 247421
gis_id                191320
landlord_name         191168
int_cond              608429
ext_cond              568442
overall_cond 

# Feature Extract